In [2]:
import graphlab

In [3]:
sales = graphlab.SFrame('kc_house_data.gl')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1539313465.log


This non-commercial license of GraphLab Create for academic use is assigned to r38411@yahoo.com and will expire on August 31, 2019.


In [4]:
graphlab.canvas.set_target('ipynb')
sales.print_rows(num_rows=3)

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000.0 |   3.0    |    2.25   |
| 5631500400 | 2015-02-25 00:00:00+00:00 | 180000.0 |   2.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
|    2570.0   |   7242   |   2    |     0      |  0   |     3     |   7   |    2170    |
|    770.0    | 

In [6]:
train_data, test_data = sales.random_split(.8, seed=0)

In [21]:
def simple_linear_regression(input_feature, output):
    x = graphlab.SArray(input_feature)
    y = graphlab.SArray(output)
    N = len(x)
    slope = 1.0 * (sum(x*y)-sum(x)*sum(y)*1.0/N)/(sum(x*x)-sum(x)*sum(x)*1.0/N)
    intercept = y.mean() - slope*x.mean()
    return (intercept, slope)

In [9]:
input_feature = train_data['sqft_living']
output = train_data['price']
(squarefeet_intercept, squarefeet_slope) = simple_linear_regression(input_feature, output)
print squarefeet_intercept, squarefeet_slope

-47116.0765749 281.958838568


# Question 3:
# squarefeet_intercept = -47116.07, squarefeet_slope = 281.958

In [10]:
## Question 5:

In [48]:
def get_regression_predictions(input_feature, intercept, slope):
    x = graphlab.SArray(input_feature)
    y = intercept + slope * x
    return y

In [24]:
## Question 6

In [49]:
input_feature = [2650]
print get_regression_predictions(input_feature, squarefeet_intercept, squarefeet_slope)

[700074.8456294581]


# Question 6: 
# for 2650 sqft, the predicted price is: 700074.8456

In [72]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    actual_output = graphlab.SArray(output)
    predicted_output = get_regression_predictions(input_feature, intercept, slope)
    #print 'my predicted_output = ', predicted_output
    RSS = sum((actual_output-predicted_output)**2)
    return RSS

In [73]:
def test_func():
    i_feature = [0,1,2,3,4]
    out = [1,3,7,13,21]
    (w0, w1) = simple_linear_regression(i_feature, out)
    print w0, w1
    print 'the actual output inside test_funct = ', out
    print 'the predicted output = ', get_regression_predictions(i_feature, w0, w1)
    print 'the RSS = ', get_residual_sum_of_squares(i_feature, out, w0, w1)
    print 'test inverse regression'
    predict_input = inverse_regression_predictions(out, w0, w1)
    print 'predict_input = ', predict_input
test_func()

-1.0 5.0
the actual output inside test_funct =  [1, 3, 7, 13, 21]
the predicted output =  [-1.0, 4.0, 9.0, 14.0, 19.0]
the RSS =  14.0
test inverse regression
predict_input =  [0.4, 0.8, 1.6, 2.8, 4.4]


In [63]:
## Question 8: get the RSS for training data

In [75]:
input_feature = train_data['sqft_living']
output = train_data['price']
RSS = get_residual_sum_of_squares(input_feature, output, squarefeet_intercept, squarefeet_slope)
print 'RSS of the training data = ', RSS

RSS of the training data =  1.20191835632e+15


# Question 8:
# RSS of the training data = 1.2019e+15

In [68]:
## question 9: inverse function

In [69]:
def inverse_regression_predictions(output, intercept, slope):
    output_val = graphlab.SArray(output)
    predicted_input = (output_val - intercept)/slope
    return predicted_input

In [74]:
## Question 10:

In [76]:
out1 = [800000]
print 'the predicted_input of 800K = ', inverse_regression_predictions(out1, squarefeet_intercept,
                                                                      squarefeet_slope)

the predicted_input of 800K =  [3004.396247615945]


# Question 10: 
# the predicted sqft of the housing of 800K = 3004.39 sqft

In [78]:
input_feature = train_data['bedrooms']
output = train_data['price']
(bedroom_intercept, bedroom_slope) = simple_linear_regression(input_feature, output)
print bedroom_intercept, bedroom_slope

109473.180469 127588.952175


# Question 11: bedroom model
# bedroom_intercept = 109473.18, bedroom_slope = 127588.95

In [81]:
input_feature = train_data['sqft_living']
output = train_data['price']
RSS_sqft = get_residual_sum_of_squares(input_feature, output, 
                                       squarefeet_intercept, squarefeet_slope)
input_feature = train_data['bedrooms']
RSS_bedroom = get_residual_sum_of_squares(input_feature, output,
                                          bedroom_intercept, bedroom_slope)
print 'RSS of sqft model = ', RSS_sqft
print 'RSS of bedroom model = ', RSS_bedroom

RSS of sqft model =  1.20191835632e+15
RSS of bedroom model =  2.14324449423e+15


In [82]:
# Question 13: RSS
# RSS of sqft model = 1.2019e15
# RSS of bedroom model = 2.1432e15
# therefore, RSS of sqft model has a smaller RSS

In [153]:
def gradient_linear_regression(input, output, init_intercept, init_slope, step_size, tol):
    # compute the predicted values given current slope and intercept
    input_feature = graphlab.SArray(input)
    actual_output = graphlab.SArray(output)
    cur_intercept = init_intercept
    cur_slope = init_slope
    error = tol * 100.0
    num_iter = 0
    while (error > tol) and (num_iter < 200):
        predicted_output = get_regression_predictions(input_feature, cur_intercept, cur_slope)
        grad_0 = sum(predicted_output-actual_output)
        grad_1 = sum((predicted_output-actual_output)*input_feature)
        cur_intercept = cur_intercept - grad_0*step_size
        cur_slope = cur_slope - grad_1*step_size
        error = ((grad_0*grad_0) + (grad_1*grad_1))**0.5
        num_iter = num_iter + 1
        #print 'inside gradient function:'
        #print 'predicted_output = ', predicted_output
        print 'new intercept = ', cur_intercept, ', new slope = ', cur_slope
        print 'magnitude of grad = ', error
        print 'grad0 = ', grad_0, 'grad1 = ', grad_1
        print 'num of iteration = ', num_iter
        print
    return cur_intercept, cur_slope

In [148]:
def test_func2():
    i_feature = [0,1,2,3,4]
    out = [1,3,7,13,21]
    init_slope = 0.0
    init_intercept = 0.0
    step_size = 0.05
    tolerance = 0.01
    (w0, w1) = gradient_linear_regression(i_feature, out, init_intercept,
                                          init_slope, step_size, tolerance)
    print w0, w1
    print 'the actual output inside test_funct = ', out
    (predict_intercept, predict_slope) = gradient_linear_regression(i_feature, out,
                                                                 init_intercept, init_slope,
                                                                 step_size, tolerance)

test_func2()

new intercept =  2.25 , new slope =  7.0
magnitude of grad =  147.054411699
grad0 =  -45.0 grad1 =  -140.0
num of iteration =  1

new intercept =  0.4375 , new slope =  2.375
magnitude of grad =  99.3494464001
grad0 =  36.25 grad1 =  92.5
num of iteration =  2

new intercept =  1.390625 , new slope =  5.59375
magnitude of grad =  67.1380632075
grad0 =  -19.0625 grad1 =  -64.375
num of iteration =  3

new intercept =  0.49609375 , new slope =  3.5078125
magnitude of grad =  45.3930453313
grad0 =  17.890625 grad1 =  41.71875
num of iteration =  4

new intercept =  0.8681640625 , new slope =  4.998046875
magnitude of grad =  30.7196016242
grad0 =  -7.44140625 grad1 =  -29.8046875
num of iteration =  5

new intercept =  0.402099609375 , new slope =  4.06689453125
magnitude of grad =  20.8255685323
grad0 =  9.3212890625 grad1 =  18.623046875
num of iteration =  6

new intercept =  0.518127441406 , new slope =  4.76550292969
magnitude of grad =  14.1635610236
grad0 =  -2.32055664062 grad1 = 

In [154]:
input_feature = train_data['sqft_living']
output = train_data['price']
init_slope = 0.0
init_intercept=0.0
step_size = 1.0e-11
tolerance = 10000
print squarefeet_intercept, squarefeet_slope
(w0, w1) = gradient_linear_regression(input_feature, output,
                                     init_intercept, init_slope,
                                     step_size, tolerance)
print w0, w1

-47116.0765749 281.958838568
new intercept =  0.09376349465 , new slope =  236.662568479
magnitude of grad =  2.36662587054e+13
grad0 =  -9376349465.0 grad1 =  -2.36662568479e+13
num of iteration =  1

new intercept =  0.10195160344 , new slope =  260.382152625
magnitude of grad =  2.37195855586e+12
grad0 =  -818810878.99 grad1 =  -2.37195841453e+12
num of iteration =  2

new intercept =  0.101562891109 , new slope =  262.759456223
magnitude of grad =  2.37730362961e+11
grad0 =  38871233.126 grad1 =  -2.37730359783e+11
num of iteration =  3

new intercept =  0.100314564098 , new slope =  262.997722743
magnitude of grad =  23826979004.1
grad0 =  124832701.074 grad1 =  -23826651994.4
num of iteration =  4

new intercept =  0.098980081958 , new slope =  263.021603569
magnitude of grad =  2391808320.13
grad0 =  133448213.984 grad1 =  -2388082623.03
num of iteration =  5

new intercept =  0.0976369649266 , new slope =  263.023997518
magnitude of grad =  274498785.82
grad0 =  134311703.146 g